In [46]:
import numpy as np
import pandas
from functools import reduce

In [3]:
def getPhysicalAttributes(league,season):
    import numpy as np
    import pandas as pd
    from functools import reduce
    from nba_api.stats.endpoints import draftcombineplayeranthro, draftcombinedrillresults
    
    draft_log_anthro = draftcombineplayeranthro.DraftCombinePlayerAnthro(league_id = league, season_year = season)
    draft_log_com_results = draftcombinedrillresults.DraftCombineDrillResults(league_id = league, season_year = season)
    
    df_anthro = draft_log_anthro.get_data_frames()[0]
    df_com_results = draft_log_com_results.get_data_frames()[0]
    df_anthro = df_anthro.replace(to_replace='None', value=np.nan).dropna(how='all',subset=['HEIGHT_WO_SHOES', 'HEIGHT_WO_SHOES_FT_IN', 'HEIGHT_W_SHOES', 'HEIGHT_W_SHOES_FT_IN', 
                                                                                  'WEIGHT', 'WINGSPAN', 'WINGSPAN_FT_IN', 'STANDING_REACH', 'STANDING_REACH_FT_IN', 
                                                                                  'BODY_FAT_PCT', 'HAND_LENGTH', 'HAND_WIDTH'])
    df_com_results = df_com_results.replace(to_replace='None', value=np.nan).dropna(how='all',subset=['STANDING_VERTICAL_LEAP', 'MAX_VERTICAL_LEAP', 'LANE_AGILITY_TIME', 
                                                                                            'MODIFIED_LANE_AGILITY_TIME', 'THREE_QUARTER_SPRINT', 'BENCH_PRESS'])
    
    #df_anthro = df_anthro.dropna()
    #df_com_results = df_com_results.dropna()#
    
    Ids = df_com_results["PLAYER_ID"].to_numpy()
    df_anthro = df_anthro.loc[df_anthro['PLAYER_ID'].isin(Ids)]
    
    final_df = pd.merge(left=df_com_results, right=df_anthro, left_on='PLAYER_ID', right_on='PLAYER_ID')
    final_df = final_df.drop(['TEMP_PLAYER_ID_y','FIRST_NAME_y','LAST_NAME_y','PLAYER_NAME_y','POSITION_y'], 1)
    
    draft_year = [season] * len(final_df)
    final_df['DRAFT_CLASS'] = draft_year
    
    return final_df
    

#df = getPhysicalAttributes('00','2000')
#display(df, len(df))
#df = getPhysicalAttributes('00','2016')
#display(df, len(df))
df = getPhysicalAttributes('00','2019')
#display(df, len(df))



In [5]:
#pd.concat([s1, s2], ignore_index=True)

def getData(season, league = '00'):
    import numpy as np
    import pandas as pd
    if len(season) > 1:
        initial = 0
        for year in season:
            if initial == 0:
                current_df = getPhysicalAttributes(league,year)
                initial = 1
            else:
                next_df = getPhysicalAttributes(league,year)
                current_df = pd.concat([current_df, next_df], ignore_index=True)
                
            
    else:
        current_df = getPhysicalAttributes(league,season[0])
    
    data = current_df
        
    return data

draft_years = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016']
data = getData(draft_years)
data.to_csv('draft_data_2000_2016.csv', index=False)
new_data = data.dropna(how='any',axis = 1)
new_data.to_csv('draft_data_2000_2016_no_empty_cells.csv', index=False)
#display(new_data)

In [22]:
def getDraftIDs(season = '2021', league = '00'):
    import numpy as np
    import pandas
    from nba_api.stats.endpoints import drafthistory
    lastDraft = drafthistory.DraftHistory(season_year_nullable = season, league_id = league)
    data = lastDraft.get_data_frames()[0]
    player_ids = data["PERSON_ID"].to_numpy()
    return player_ids

getDraftIDs(season ='2019')

array([1629627, 1629630, 1629628, 1629631, 1629636, 1629633, 1629632,
       1629637, 1629060, 1629629, 1629661, 1629023, 1629639, 1629641,
       1629635, 1629643, 1629638, 1629048, 1629677, 1629680, 1629634,
       1629684, 1629647, 1629660, 1629642, 1629685, 1629662, 1629673,
       1629640, 1629645, 1629651, 1629644, 1629035, 1628981, 1629712,
       1628998, 1629686, 1629655, 1629346, 1629713, 1629672, 1629678,
       1629669, 1629626, 1629676, 1629659, 1629649, 1629611, 1629683,
       1629714, 1629682, 1629667, 1629625, 1629621, 1629657, 1628986,
       1629648, 1629671, 1629608, 1629079], dtype=int64)